In [20]:
pip install plotly panel

Note: you may need to restart the kernel to use updated packages.


In [21]:
import pandas as pd
import numpy as np
import plotly.express as px
import panel as pn

In [22]:
# Read transactions_2022_2023_categorized.csv
df = pd.read_csv('combined_transactions_categories.csv')
# Add year and month columns
df['Year'] = pd.to_datetime(df['Date']).dt.year
df['Month'] = pd.to_datetime(df['Date']).dt.month
df['Month Name'] = pd.to_datetime(df['Date']).dt.strftime("%b")
# Remove "Transaction" and "Transaction vs category" columns
df = df.drop(columns=['Transaction', 'Transaction vs category'])
df

,Date,Name / Description,Expense/Income,Amount (EUR),Category,Year,Month,Month Name
0,2023-11-04,Salary,Income,4.5,Income,2023,11,Nov
1,2023-11-05,YouTube Revenue,Income,1.2,Business,2023,11,Nov
2,2023-11-06,Anime Streaming Subscriptions,Expense,50.0,Entertainment,2023,11,Nov
3,2023-11-07,Book Purchases,Expense,80.0,Entertainment,2023,11,Nov
4,2023-11-08,Rent,Expense,1.2,Housing,2023,11,Nov
...,...,...,...,...,...,...,...,...
139,2024-05-08,"Event Tickets (Concerts, Sports)",Expense,150.0,NaN,2024,5,May
140,2024-05-09,Seasonal Decorations,Expense,30.0,NaN,2024,5,May
141,2024-05-10,Custom Anime Art,Expense,60.0,NaN,2024,5,May
142,2024-05-11,Pet Insurance,Expense,70.0,NaN,2024,5,May


In [23]:
# For Income rows, assign Name / Description to Category
df['Category'] = np.where(df['Expense/Income'] == 'Income', df['Name / Description'], df['Category'])

In [24]:
import pandas as pd
import plotly.express as px

def make_pie_chart(df, year, label):
    # Filter the dataset for the specified expense/income and year
    sub_df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]

    # Check if the filtered dataframe is empty
    if sub_df.empty:
        raise ValueError(f"No data available for {label} in year {year}")

    color_scale = px.colors.qualitative.Set2
    
    pie_fig = px.pie(sub_df, values='Amount (EUR)', names='Category', color_discrete_sequence=color_scale)
    pie_fig.update_traces(textposition='inside', direction='clockwise', hole=0.3, textinfo="label+percent")

    # Calculate totals
    expense_df = df[(df['Expense/Income'] == 'Expense') & (df['Year'] == year)]
    income_df = df[(df['Expense/Income'] == 'Income') & (df['Year'] == year)]

    # Ensure Amount (EUR) is numeric
    expense_df['Amount (EUR)'] = pd.to_numeric(expense_df['Amount (EUR)'], errors='coerce')
    income_df['Amount (EUR)'] = pd.to_numeric(income_df['Amount (EUR)'], errors='coerce')

    total_expense = expense_df['Amount (EUR)'].sum()
    total_income = income_df['Amount (EUR)'].sum()

    if pd.isna(total_income) or pd.isna(total_expense):
        raise ValueError("Total income or expense calculation returned NaN")

    # Ensure totals are numeric before rounding
    if label == 'Expense':
        total_text = "€ " + str(round(total_expense))

        # Calculate saving rate
        if total_income > 0:
            saving_rate = round((total_income - total_expense) / total_income * 100)
            saving_rate_text = ": Saving rate " + str(saving_rate) + "%"
        else:
            saving_rate_text = ": Saving rate N/A"
    else:
        saving_rate_text = ""
        total_text = "€ " + str(round(total_income))

    pie_fig.update_layout(
        uniformtext_minsize=10,
        uniformtext_mode='hide',
        title=dict(text=label + " Breakdown " + str(year) + saving_rate_text),
        annotations=[
            dict(
                text=total_text,
                x=0.5, y=0.5, font_size=12,
                showarrow=False
            )
        ]
    )

    return pie_fig

# Example usage
try:
    income_pie_fig_2024 = make_pie_chart(df, 2024, 'Income')
    income_pie_fig_2024.show()  # Use .show() to display the figure
except Exception as e:
    print(f"Error: {e}")


C:\Users\dell\AppData\Local\Temp\ipykernel_18688\1756192668.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dell\AppData\Local\Temp\ipykernel_18688\1756192668.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [25]:
income_pie_fig_2024 = make_pie_chart(df, 2024, 'Income')
income_pie_fig_2024

C:\Users\dell\AppData\Local\Temp\ipykernel_18688\1756192668.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dell\AppData\Local\Temp\ipykernel_18688\1756192668.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
def make_monthly_bar_chart(df, year, label):
    df = df[(df['Expense/Income'] == label) & (df['Year'] == year)]
    total_by_month = (df.groupby(['Month', 'Month Name'])['Amount (EUR)'].sum()
                        .to_frame()
                        .reset_index()
                        .sort_values(by='Month')  
                        .reset_index(drop=True))
    if label == "Income":
        color_scale = px.colors.sequential.YlGn
    if label == "Expense":
        color_scale = px.colors.sequential.OrRd
    
    bar_fig = px.bar(total_by_month, x='Month Name', y='Amount (EUR)', text_auto='.2s', title=label+" per month", color='Amount (EUR)', color_continuous_scale=color_scale)
    # bar_fig.update_traces(marker_color='lightslategrey')
    
    return bar_fig

In [27]:
income_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Income')
income_monthly_2024

In [28]:
# Pie charts
#income_pie_fig_2023 = make_pie_chart(df, 2023, 'Income')
#expense_pie_fig_2023 = make_pie_chart(df, 2023, 'Expense')  
income_pie_fig_2024 = make_pie_chart(df, 2024, 'Income')
expense_pie_fig_2024 = make_pie_chart(df, 2024, 'Expense')

# Bar charts
#income_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Income')
#expense_monthly_2023 = make_monthly_bar_chart(df, 2023, 'Expense')
income_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Income')
expense_monthly_2024 = make_monthly_bar_chart(df, 2024, 'Expense')

# Create tabs
tabs = pn.Tabs(
                        #('2023', pn.Column(pn.Row(income_pie_fig_2023, expense_pie_fig_2023),
                                                #pn.Row(income_monthly_2023, expense_monthly_2023))),
                        ('2024', pn.Column(pn.Row(income_pie_fig_2024, expense_pie_fig_2024),
                                                pn.Row(income_monthly_2024, expense_monthly_2024))
                        )
                )
tabs.show()

C:\Users\dell\AppData\Local\Temp\ipykernel_18688\1756192668.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dell\AppData\Local\Temp\ipykernel_18688\1756192668.py:23: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\dell\AppData\Local\Temp\ipykernel_18688\1756192668.py:22: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/panda

Launching server at http://localhost:54399


In [29]:
# Dashboard template
template = pn.template.FastListTemplate(
    title='Personal Finance Dashboard',
    sidebar=[pn.pane.Markdown("# Income Expense analysis"), 
             pn.pane.Markdown("Overview of income and expense based on my bank transactions. Categories are obtained using local LLMs."),
             pn.pane.PNG("picture.png", sizing_mode="scale_both")
             ],
    main=[pn.Row(pn.Column(pn.Row(tabs)
                           )
                ),
                ],
    # accent_base_color="#88d8b0",
    header_background="#c0b9dd",
)

template.show()

Launching server at http://localhost:54400
